In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action = 'ignore')

In [4]:
amzn = pd.read_csv("dataset/AMZN_2006-01-01_to_2018-01-01.csv")
googl = pd.read_csv("dataset/GOOGL_2006-01-01_to_2018-01-01.csv")
ibm = pd.read_csv("dataset/IBM_2006-01-01_to_2018-01-01.csv")
msft = pd.read_csv("dataset/MSFT_2006-01-01_to_2018-01-01.csv")
aapl = pd.read_csv("dataset/AAPL_2006-01-01_to_2018-01-01.csv")
intc = pd.read_csv("dataset/INTC_2006-01-01_to_2018-01-01.csv")
vz = pd.read_csv("dataset/VZ_2006-01-01_to_2018-01-01.csv")
amd = pd.read_csv("dataset/AMD.csv")
nvda = pd.read_csv("dataset/nvidia_all_time_daily_stock_prices.csv")
aaba = pd.read_csv("dataset/AABA_2006-01-01_to_2018-01-01.csv")

In [5]:
amd['Date'] = pd.to_datetime(amd['Date'])
nvda['Date'] = pd.to_datetime(nvda['Date'])
start_date = pd.to_datetime('2006-01-03')
end_date = pd.to_datetime('2017-12-29')
amd_crop = amd[(amd['Date'] >= start_date) & (amd['Date'] <= end_date)]
amd_crop = amd_crop.reset_index(drop=True)
amd_crop['Close'] = amd_crop['Close'].round(2)
nvda_crop = nvda[(nvda['Date'] >= start_date) & (nvda['Date'] <= end_date)]
nvda_crop = nvda_crop.reset_index(drop = True)
nvda_crop['Close'] = nvda_crop['Close'].round(2)

In [45]:
data = pd.concat([amzn[['Date','Close']], googl["Close"], msft["Close"], aapl["Close"], intc['Close'], 
                  nvda_crop['Close'], ibm["Close"], vz['Close'], aaba['Close'], amd_crop['Close']], axis = 1)
data.columns = ['Date', 'amzn', 'googl', 'msft', 'aapl', 'intc','nvda', 'ibm', 'vz', 'aaba', 'amd']
data = data.drop(data.index[-1])
data['Date'] = pd.to_datetime(data['Date'])
cols = list(data)[1:11]

train_data = data[:-100]
test_data = data[-100:]

test_data.head(5)

,Date,amzn,googl,msft,aapl,intc,nvda,ibm,vz,aaba,amd
2919,2017-08-09,982.01,940.08,72.47,161.06,36.59,42.58,142.11,48.60,59.77,13.11
2920,2017-08-10,956.92,923.59,71.41,155.32,36.14,43.03,141.77,48.22,58.03,12.83
2921,2017-08-11,967.99,930.09,72.50,157.48,35.87,41.19,141.84,48.02,57.92,12.12
2922,2017-08-14,983.30,938.93,73.59,159.85,36.34,38.99,141.84,48.06,58.87,12.23
2923,2017-08-15,982.74,938.08,73.22,161.60,36.00,42.10,142.32,48.78,59.99,12.76


In [46]:
train_date = train_data['Date']
test_date = test_data['Date']
train_data = train_data[cols].astype(float)
test_data = test_data[cols].astype(float)
np.shape(train_data)

train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   amzn    2919 non-null   float64
 1   googl   2919 non-null   float64
 2   msft    2919 non-null   float64
 3   aapl    2919 non-null   float64
 4   intc    2919 non-null   float64
 5   nvda    2919 non-null   float64
 6   ibm     2919 non-null   float64
 7   vz      2919 non-null   float64
 8   aaba    2919 non-null   float64
 9   amd     2919 non-null   float64
dtypes: float64(10)
memory usage: 228.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 2919 to 3018
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   amzn    100 non-null    float64
 1   googl   100 non-null    float64
 2   msft    100 non-null    float64
 3   aapl    100 non-null    float64
 4   intc    100 non-null    float64
 5   nvda    100 non-null    float64
 

In [47]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc = sc.fit(train_data)
scaled_train_data = sc.transform(train_data)

In [48]:
x_train = []
y_train = []

n_future = 1
n_past = 30

for i in range(n_past, len(scaled_train_data) - n_future + 1):
    x_train.append(scaled_train_data[i - n_past:i, 0: train_data.shape[1]])
    y_train.append(scaled_train_data[i + n_future - 1 :i + n_future, 0 : train_data.shape[1]])

x_train, y_train = np.array(x_train), np.array(y_train)

In [49]:
y_train.shape[2]

10

In [50]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam

In [51]:
model = Sequential()
model.add(LSTM(128, activation = 'relu', input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = True))
model.add(LSTM(128, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(64, activation = 'relu', return_sequences = False))
model.add(Dense(y_train.shape[2]))
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'mse')
'''
model.add(LSTM(64, activation = 'relu', input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = True))
model.add(LSTM(32, activation = 'relu', return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[2]))
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'mse')
'''
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 128)           71168     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           131584    
                                                                 
 dropout_3 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 252810 (987.54 KB)
Trainable params: 252810 (987.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [55]:
history = model.fit(x_train, y_train, epochs = 10, shuffle = False, validation_split = 0.1)

Epoch 1/10
82/82 [==============================] - 3s 39ms/step - loss: 4.5946 - accuracy: 0.5823 - val_loss: 1.8970 - val_accuracy: 0.0000e+00
Epoch 2/10
82/82 [==============================] - 3s 39ms/step - loss: 0.3439 - accuracy: 0.5773 - val_loss: 1.6058 - val_accuracy: 0.0000e+00
Epoch 3/10
82/82 [==============================] - 4s 46ms/step - loss: 0.4608 - accuracy: 0.3446 - val_loss: 1.1205 - val_accuracy: 0.2907
Epoch 4/10
82/82 [==============================] - 4s 46ms/step - loss: 0.2975 - accuracy: 0.5669 - val_loss: 1.9066 - val_accuracy: 0.2907
Epoch 5/10
82/82 [==============================] - 4s 46ms/step - loss: 0.2587 - accuracy: 0.4719 - val_loss: 1.3830 - val_accuracy: 0.0000e+00
Epoch 6/10
82/82 [==============================] - 4s 46ms/step - loss: 0.1635 - accuracy: 0.6423 - val_loss: 1.2857 - val_accuracy: 0.0000e+00
Epoch 7/10
82/82 [==============================] - 4s 46ms/step - loss: 0.9336 - accuracy: 0.6369 - val_loss: 1.8990 - val_accuracy: 0.24